In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf

# Load the labeled MCQs dataset from an Excel file
df = pd.read_excel('/content/my2.xlsx')
mcqs = df.iloc[:, 0].tolist()
labels = df.iloc[:, 6].tolist()

# Split the dataset into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(mcqs, labels, test_size=0.2, random_state=42)

# Encode the labels using LabelEncoder
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

# Initialize the BERT tokenizer and encode the texts
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels))

# Load the pre-trained BERT model for sequence classification
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss=model.hf_compute_loss, metrics=['accuracy'])

# Train the model
model.fit(train_dataset.shuffle(1000).batch(16), epochs=5, batch_size=16)

# Evaluate the model on the test dataset
_, accuracy = model.evaluate(test_dataset.batch(16), verbose=0)
print('Test Accuracy:', accuracy)


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_99', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/5
50/50 [==============================] - 467s 9s/step - loss: 1.0556 - accuracy: 0.4161
Epoch 2/5
50/50 [==============================] - 447s 9s/step - loss: 0.9477 - accuracy: 0.4994
Epoch 3/5
50/50 [==============================] - 444s 9s/step - loss: 0.9148 - accuracy: 0.5120
Epoch 4/5
50/50 [==============================] - 450s 9s/step - loss: 0.8948 - accuracy: 0.5322
Epoch 5/5
50/50 [==============================] - 443s 9s/step - loss: 0.8802 - accuracy: 0.5485
Test Accuracy: 0.4673366844654083
